In [1]:
import numpy as np
from PIL import Image
import requests
from io import BytesIO

def load_and_preprocess_image(url, img_size=(224, 224)):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    img = img.resize(img_size)
    img = np.array(img) / 255.0  # normalize to [0,1] range
    return img



In [2]:
import pandas as pd
import re

df = pd.read_csv('menu_data.csv')

# Load images
images = np.array([load_and_preprocess_image(url) for url in df['img_url']])

# Extract numerical price from the price string
prices = np.array([int(re.search(r'\d+', price).group()) for price in df['price']])


In [3]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=images.shape[1:])
x = base_model.output
x = Flatten()(x)
x = Dense(1)(x)  # Only one output node, because we're doing regression
model = Model(inputs=base_model.input, outputs=x)

model.compile(optimizer=Adam(), loss='mean_squared_error')  # Regression task, so we use MSE loss


94765736/94765736 [==============================] - 4s 0us/step


In [4]:
model.fit(images, prices, epochs=10, validation_split=0.2)


Epoch 1/10
5/5 [==============================] - 109s 14s/step - loss: 166026.9531 - val_loss: 1361185.0000
Epoch 2/10
5/5 [==============================] - 62s 12s/step - loss: 49125.7773 - val_loss: 297498.7188
Epoch 3/10
5/5 [==============================] - 64s 13s/step - loss: 49345.6602 - val_loss: 136936.9062
Epoch 4/10
5/5 [==============================] - 62s 12s/step - loss: 27781.0312 - val_loss: 449345.0000
Epoch 5/10
5/5 [==============================] - 62s 12s/step - loss: 20525.7051 - val_loss: 352028.8125
Epoch 6/10
5/5 [==============================] - 64s 12s/step - loss: 21443.8809 - val_loss: 802425.1875
Epoch 7/10
5/5 [==============================] - 68s 13s/step - loss: 11151.7695 - val_loss: 632714.7500
Epoch 8/10
5/5 [==============================] - 79s 16s/step - loss: 10598.4443 - val_loss: 1266772.5000
Epoch 9/10
5/5 [==============================] - 75s 14s/step - loss: 10459.5732 - val_loss: 1207523.8750
Epoch 10/10
5/5 [========================

In [5]:
def predict_price(model, img_url):
    # 画像のダウンロードと前処理
    img = load_and_preprocess_image(img_url)
    img = np.expand_dims(img, axis=0)  # モデルに合わせて次元を追加

    # 価格予測
    predicted_price = model.predict(img)
    
    return predicted_price[0][0]


In [9]:
!pip install optuna

import optuna
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

def create_model(trial, input_shape):
    model = Sequential()

    # 畳み込み層
    model.add(Conv2D(filters=trial.suggest_int("filters", 32, 128), kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=2))

    # 全結合層
    model.add(Flatten())
    model.add(Dense(units=trial.suggest_int("units", 32, 128), activation='relu'))

    # 出力層
    model.add(Dense(1))

    return model

def objective(trial, X, y):
    model = create_model(trial, X.shape[1:])
    model.compile(optimizer='adam', loss='mean_squared_error')

    history = model.fit(X, y, epochs=5, validation_split=0.2, batch_size=32, verbose=0)

    return history.history["val_loss"][-1]

import numpy as np

# Load images and prices data
images = np.load('path_to_your_images_file.npy')
prices = np.load('path_to_your_prices_file.npy')

# Now, you can use images and prices in your optimization
study = optuna.create_study(direction="minimize")
study.optimize(lambda trial: objective(trial, images, prices), n_trials=100)


FileNotFoundError: [Errno 2] No such file or directory: 'path_to_your_images_file.npy'

In [9]:
new_img_url = "https://img.pretty-online.jp/wp-content/uploads/2022/07/21140818/gourmet_shinosakalunch_3.jpeg.webp"  # 予測したい画像のURL
predicted_price = predict_price(model, new_img_url)

print("Predicted price: ", predicted_price)



1/1 [==============================] - 0s 143ms/step
Predicted price:  3497.5671
